# 📊YOLOv8

### In dit notebook vergelijken we onderstaande YOLO-modellen
- YOLOv8n
 - YOLOv8s
  - YOLOv8m
  ##### Daarna bepalen we welk model het beste werkt en de verschillende klasses op de gevel detecteert. Elk model wordt getraind en getest op dezelfde dataset, met verschillende epochs.

 ##### Om de modellen te beoordelen, gebruiken we onder andere IOU (Intersection over Union) om te kijken hoe nauwkeurig de voorspellingen zijn. Daarnaast kijken we met behulp van Multi-Criteria Decision-Making (MCDM) en kiezen we het model dat het beste scoort op zowel prestaties als efficiëntie.

&nbsp;


&nbsp;
&nbsp;


 We trainen niet met zwaardere modellen zoals **YOLOv8x**, omdat deze meer data vereisen. Voor onze beperkte dataset zou dat kunnen leiden tot overfitting en onnodige rekentijd.


&nbsp;


#### Import en instalatie:

In [41]:
from ultralytics import YOLO
import cv2
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

 Voor het trainen van deze modellen is het **sterk aanbevolen om een NVIDIA GPU te gebruiken**. YOLOv8 is gebouwd op PyTorch, dat gebruikmaakt van **CUDA**  een technologie die alleen werkt met **NVIDIA GPU’s**.

Zonder GPU werkt het model nog steeds, maar:
- Training duurt **veel langer**
- Inference (testen) is **trager**
- Sommige modellen kunnen geheugenproblemen veroorzaken op CPU



###  Wat heb je nodig?
- Een systeem met een **NVIDIA GPU met CUDA-ondersteuning**
- Of gebruik een platform zoals **Google Colab**, waar gratis GPU’s beschikbaar zijn



&nbsp;

#### Beschikbare NVIDIA GPU’s:

In [ ]:
!nvidia-smi

Checken waar de model draait (GPU of CPU):

In [ ]:
# ✅ Check of GPU beschikbaar is
device = 'GPU' if torch.cuda.is_available() else 'cpu'
print(f"Model draait op: {device}")


#### Dataset laden

In [ ]:
# Onze Yolo formaat dataset laden
data_path = r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\data.yaml'

> ⚠️ **Let op:** Het pad is afhankelijk van je eigen bestandslocatie. Pas dit dus aan naar het juiste pad op jouw computer.

&nbsp;


&nbsp;


### 🚀 We starten met het trainen van het model **YOLOv8n** met 50 epochs


> ⚠️ We trainen **niet met zwaardere modellen zoals YOLOv8x**, omdat deze meer data en GPU-geheugen vereisen. Voor onze beperkte dataset zou dat kunnen leiden tot overfitting en onnodige rekentijd.


In [ ]:
yolov8n_nest_50epochs = YOLO('yolov8n-seg.pt')
yolov8n_nest_50epochs.train(
    data= data_path,
    epochs=50,
    device=0,
    pretrained=False,
    project=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen',
    name='yolov8n_nest_scratch_50epochs')

In [ ]:
yolov8n_nest_50epochs.save('yolov8n_nest_50epochs')


### 🔁 Tweede training: YOLOv8n nogmaals met 40 epochs


In [ ]:
yolov8n_nest_40epochs = YOLO('yolov8n-seg.pt')
yolov8n_nest_40epochs.train(
    data= data_path,
    epochs=50,
    device=0,
    pretrained=False,
    project=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen',
    name='yolov8n_nest_scratch_40epochs')
yolov8n_nest_40epochs.save('yolov8n_nest_40epochs')

## Eerste evaluatie

In dit deel kijken we welk model het beste werkt op onze testbeelden.

We vergelijken onder andere:
- **IOU** – hoe goed de voorspelde objecten overeenkomen met de echte
- **mAP** – hoe nauwkeurig het model werkt in het algemeen
- **Precision en Recall** – hoe goed en volledig het model dingen herkent

We testen beide modellen (YOLOv8n met 40 en 50 epochs) op dezelfde testset, zodat we eerlijk kunnen vergelijken.

In [ ]:
# Modellen laden vanaf het juiste pad
model_40 = YOLO(r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\yolov8n_nest_40epochs.pt')
model_50 = YOLO(r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\yolov8n_nest_50epochs.pt')


##### Modellen runnen op dataset

In [ ]:
results_40 = model_40.predict(
    source=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images',
    save=False
)

results_50 = model_50.predict(
    source=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images',
    save=False
)


In [ ]:
metrics_40 = model_40.val(data=data_path, split='test')
metrics_50 = model_50.val(data=data_path, split='test')

In [ ]:
print("📊 Evaluatieresultaten YOLOv8n")

# Model met 40 epochs
print("\n🔹 Model 40 epochs:")
print(f"mAP@0.5:       {metrics_40.box.map50:.4f}")
print(f"mAP@0.5:0.95:  {metrics_40.box.map:.4f}")
print(f"Precision:     {metrics_40.box.mp:.4f}")
print(f"Recall:        {metrics_40.box.mr:.4f}")

# Model met 50 epochs
print("\n🔹 Model 50 epochs:")
print(f"mAP@0.5:       {metrics_50.box.map50:.4f}")
print(f"mAP@0.5:0.95:  {metrics_50.box.map:.4f}")
print(f"Precision:     {metrics_50.box.mp:.4f}")
print(f"Recall:        {metrics_50.box.mr:.4f}")

# Vergelijking
print("\n✅ Vergelijking:")
if metrics_50.box.map > metrics_40.box.map:
    print("→ Het 50-epoch model heeft een hogere mAP@0.5:0.95.")
else:
    print("→ Het 40-epoch model heeft een hogere mAP@0.5:0.95.")

if metrics_50.box.map50 > metrics_40.box.map50:
    print("→ Het 50-epoch model heeft een hogere mAP@0.5.")
else:
    print("→ Het 40-epoch model heeft een hogere mAP@0.5.")


### YOLOv8s Trainen

Nu stap trainen we het model **YOLOv8s**.
We gebruiken hiervoor dezelfde dataset en trainen het model gedurende **50 epochs**.

De resultaten worden opgeslagen in de map `Modellen/yolov8s_nest_scratch_50epochs`.

Na deze training zullen we ook YOLOv8m op dezelfde manier trainen, om de prestaties te vergelijken.


In [ ]:
#  Model vanaf YAML laden
model_s = YOLO('yolov8s-seg.yaml')

#  Train met 50 epochs en sla het model op in de juiste map
model_s.train(
    data=data_path,
    epochs=50,
    device=0,
    pretrained=False,
    project=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen',
    name='yolov8s_nest_scratch_50epochs'
)


In [ ]:
model_s.save('yolov8s_nest_50epochs') # model opslaan

### YOLOv8m ook Trainen

In [25]:
#  Model vanaf YAML laden
model_m = YOLO('yolov8m-seg.yaml')

#  Train met 80 epochs en sla het model op in de juiste map
model_m.train(
    data=data_path,
    epochs=80,
    device=0,
    pretrained=False,
    project=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen',
    name='yolov8m_nest_scratch_80epochs'
)


Ultralytics 8.3.155  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=80, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-seg.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_nest_scratch_80epochs2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

train: Scanning C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\train\labels.cache... 57 images, 0 backgrounds, 0 corrupt: 100%|██████████| 57/57 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.10.0 ms, read: 807.9219.6 MB/s, size: 134.0 KB)


val: Scanning C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\valid\labels.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


Plotting labels to C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen\yolov8m_nest_scratch_80epochs2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 86 weight(decay=0.0), 97 weight(decay=0.0005), 96 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen\yolov8m_nest_scratch_80epochs2
Starting training for 80 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.85it/s]


                   all          4         35      0.893        0.5      0.924       0.45      0.893        0.5      0.924      0.538
                facade          4          4      0.782        0.5      0.782      0.438      0.782        0.5      0.782      0.615
                  roof          1          1          1          0      0.995      0.124          1          0      0.995      0.249
                window          4         30      0.897          1      0.995      0.789      0.897          1      0.995      0.749
Speed: 0.3ms preprocess, 35.3ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen\yolov8m_nest_scratch_80epochs2


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F0C6E96F20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    

In [26]:
model_m.save('yolov8m_nest_80epochs.pt') # model opslaan

### ✅ Evaluatie: YOLOv8n vs YOLOv8s vs YOLOv8m

In deze stap vergelijken we de prestaties van drie YOLOv8-modellen:
- **YOLOv8n** (nano)
- **YOLOv8s** (small)
- **YOLOv8m** (medium)

We kijken naar de volgende evaluatiemetingen op dezelfde testset:
- **mAP@0.5** – nauwkeurigheid bij 0.5 IOU
- **mAP@0.5:0.95** – algemene nauwkeurigheid
- **Precision** – hoe precies de voorspellingen zijn
- **Recall** – hoe volledig het model objecten detecteert

Het doel is te bepalen welk model het beste presteert binnen de beperkingen van onze dataset.


In [27]:
# Laad de modellen
model_n = YOLO(r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\yolov8n_nest_50epochs.pt')
model_s = YOLO(r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen\yolov8s_nest_scratch_50epochs\weights\best.pt')
model_m = YOLO(r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen\yolov8m_nest_scratch_80epochs\weights\best.pt')

# Valideer op dezelfde testset
data_path = r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\data.yaml'
metrics_n = model_n.val(data=data_path, split='test')
metrics_s = model_s.val(data=data_path, split='test')
metrics_m = model_m.val(data=data_path, split='test')


Ultralytics 8.3.155  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce GTX 1080, 8192MiB)
YOLOv8n-seg summary (fused): 85 layers, 3,258,844 parameters, 0 gradients, 12.0 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1186.2192.9 MB/s, size: 163.2 KB)


val: Scanning C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\labels.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

val: C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images\rijwoning-6-_png.rf.a677db79b71b58093105ad296ef13e93.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


...

val: C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images\rijwoning-6-_png.rf.a677db79b71b58093105ad296ef13e93.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.97s/it]


                   all          4         62      0.577      0.663      0.669      0.484      0.659      0.726      0.742      0.431
                facade          4          4      0.232       0.75      0.591      0.411      0.314          1      0.995      0.399
                  roof          1          2          1      0.976      0.995      0.796          1      0.957      0.995      0.821
                  tree          1          5      0.237        0.2      0.356      0.251      0.456        0.2      0.219     0.0355
                window          4         51      0.841      0.726      0.734       0.48      0.864      0.745      0.759      0.469
Speed: 0.4ms preprocess, 108.4ms inference, 0.0ms loss, 8.8ms postprocess per image
Results saved to runs\segment\val7


In [28]:
print("📊 Evaluatie van YOLOv8n vs YOLOv8s vs YOLOv8m (alle 50 epochs)\n")

# YOLOv8n
print("🔹 YOLOv8n:")
print(f"mAP@0.5:       {metrics_n.box.map50:.4f}")
print(f"mAP@0.5:0.95:  {metrics_n.box.map:.4f}")
print(f"Precision:     {metrics_n.box.mp:.4f}")
print(f"Recall:        {metrics_n.box.mr:.4f}\n")

# YOLOv8s
print("🔹 YOLOv8s:")
print(f"mAP@0.5:       {metrics_s.box.map50:.4f}")
print(f"mAP@0.5:0.95:  {metrics_s.box.map:.4f}")
print(f"Precision:     {metrics_s.box.mp:.4f}")
print(f"Recall:        {metrics_s.box.mr:.4f}\n")

# YOLOv8m
print("🔹 YOLOv8m:")
print(f"mAP@0.5:       {metrics_m.box.map50:.4f}")
print(f"mAP@0.5:0.95:  {metrics_m.box.map:.4f}")
print(f"Precision:     {metrics_m.box.mp:.4f}")
print(f"Recall:        {metrics_m.box.mr:.4f}\n")

# Eenvoudige vergelijking
best_map = max(metrics_n.box.map, metrics_s.box.map, metrics_m.box.map)
if best_map == metrics_n.box.map:
    print("✅ Beste model op basis van mAP@0.5:0.95: YOLOv8n")
elif best_map == metrics_s.box.map:
    print("✅ Beste model op basis van mAP@0.5:0.95: YOLOv8s")
else:
    print("✅ Beste model op basis van mAP@0.5:0.95: YOLOv8m")


📊 Evaluatie van YOLOv8n vs YOLOv8s vs YOLOv8m (alle 50 epochs)

🔹 YOLOv8n:
mAP@0.5:       0.6722
mAP@0.5:0.95:  0.6045
Precision:     0.6698
Recall:        0.6514

🔹 YOLOv8s:
mAP@0.5:       0.4871
mAP@0.5:0.95:  0.2311
Precision:     0.7959
Recall:        0.2966

🔹 YOLOv8m:
mAP@0.5:       0.6690
mAP@0.5:0.95:  0.4843
Precision:     0.5774
Recall:        0.6628

✅ Beste model op basis van mAP@0.5:0.95: YOLOv8n


### YOLOv8n heeft de beste algemene nauwkeurigheid met de hoogste mAP@0.5:0.95 en een goede balans tussen precision en recall

&nbsp;
&nbsp;

### 🧪 Testresultaat op moeilijkste geveltekening: *Rijwoning 6*

In [33]:
# Zorg dat dit overeenkomt met je modeldefinitie
yolov8n_nest_50epochs = YOLO(r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\yolov8n_nest_50epochs.pt')

# Voorspel met predict() in plaats van model()
r = yolov8n_nest_50epochs.predict(
    source=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images\rijwoning-6-_png.rf.a677db79b71b58093105ad296ef13e93.jpg',
    save=False
)

# Sla de outputafbeelding op
r[0].save(filename='result_yolov8n.jpg')



image 1/1 C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images\rijwoning-6-_png.rf.a677db79b71b58093105ad296ef13e93.jpg: 224x640 2 facades, 2 roofs, 1 tree, 22 windows, 54.9ms
Speed: 1.8ms preprocess, 54.9ms inference, 7.2ms postprocess per image at shape (1, 3, 224, 640)


'result_yolov8n.jpg'

In [34]:
# Laad het YOLOv8s-model
yolov8s_nest_50epochs = YOLO(r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\Modellen\yolov8s_nest_scratch_50epochs\weights\best.pt')

# Voorspel op de afbeelding
r_s = yolov8s_nest_50epochs.predict(
    source=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images\rijwoning-6-_png.rf.a677db79b71b58093105ad296ef13e93.jpg',
    save=False
)

# Sla resultaat op
r_s[0].save(filename='result_yolov8s.jpg')


image 1/1 C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images\rijwoning-6-_png.rf.a677db79b71b58093105ad296ef13e93.jpg: 224x640 23 windows, 96.0ms
Speed: 1.6ms preprocess, 96.0ms inference, 11.2ms postprocess per image at shape (1, 3, 224, 640)


'result_yolov8s.jpg'

In [39]:
yolov8m_nest_80epochs = YOLO(r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\yolov8m_nest_80epochs.pt')

# Voorspel op een testafbeelding
r_m = yolov8m_nest_80epochs.predict(
    source=r'C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images\rijwoning-6-_png.rf.a677db79b71b58093105ad296ef13e93.jpg',
    save=False
)

# Sla resultaatafbeelding op
r_m[0].save(filename='result_yolov8m.jpg')


image 1/1 C:\Users\karoo\CLONED\Nest-Project\Nest-Project\nest_data\test\images\rijwoning-6-_png.rf.a677db79b71b58093105ad296ef13e93.jpg: 224x640 25 windows, 76.5ms
Speed: 1.7ms preprocess, 76.5ms inference, 4.5ms postprocess per image at shape (1, 3, 224, 640)


'result_yolov8m.jpg'



Om de prestaties van de drie modellen verder te beoordelen, hebben we ze getest op één van de moeilijkste testafbeeldingen:
📄 **"rijwoning 6"** — een complexe gevel met meerdere objectklassen en een grote boom.

#### 🔍 Resultaten:

- **YOLOv8n** detecteert **alle klassen correct**: ramen, dak, gevel en raam.
- **YOLOv8s** en **YOLOv8m** detecteren **alleen de ramen**, en laten andere objecten volledig weg.


Naast de eerder berekende evaluatie (IOU) laat de outputafbeelding duidelijk zien dat **YOLOv8n in de praktijk beter presteert**.
Het model herkent meerdere objectklassen waar de andere modellen tekortschieten.

> ✅ **Conclusie:** YOLOv8n is niet alleen statistisch sterker, maar toont dit ook in de praktijk met betere detecties. Dit ondersteunt onze keuze voor YOLOv8n als het meest geschikte model voor dit project.


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Laad en toon de afbeelding
img_path = r"C:\Users\karoo\CLONED\Nest-Project\Nest-Project\foto.jpg"
img = mpimg.imread(img_path)

plt.imshow(img)
plt.axis('off')  # Geen assen tonen
plt.title("Mijn afbeelding")
plt.show()


### Gevel visualiseren

In [45]:
# Laad en toon de afbeelding
img_path = r"C:\Users\karoo\CLONED\Nest-Project\Nest-Project\foto.jpg"
img = mpimg.imread(img_path)

plt.imshow(img)
plt.axis('off')  # Geen assen tonen
plt.title("Mijn afbeelding")
plt.show()

<Figure size 640x480 with 1 Axes>